# Activity: La mañanera

Con esta actividad se pretende conocer una manera artesanal de hacer análisis de lenguaje natural. 

También sirve al propósito de introducirnos a python.

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import glob
import re
import io
import os

## Util

In [2]:
# Pass the url and the path where the html will be stored in.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
def download(url, path):
    response = requests.get(url, headers = headers)
    with open(path, "w") as f:
        f.write(response.text)

## Proof of concepts

In [12]:
download("https://presidente.gob.mx/secciones/version-estenografica/page/2/", "../data/presidente/test.txt")

In [13]:
pattern = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
re.findall(pattern, open("../data/presidente/test.txt", "r", encoding = "utf-8").read())[:3]

[('http',
  '151.394.mwp.accessdomain.com',
  '/wp-content/themes/diplomat/favicon.ico'),
 ('https', 'presidente.gob.mx', '/feed/'),
 ('https', 'presidente.gob.mx', '/xmlrpc.php')]

## Web crawling

### Fetch the pages

In [6]:
url_pagina_i = "https://presidente.gob.mx/secciones/version-estenografica/page/"
num_paginas = 117
for i in range(num_paginas): 
    download(url_pagina_i + str(i + 1) + "/", "../data/mananera/" + str(i + 1) + ".txt")

### Get the conferences' URLs from page contents

In [14]:
urls_conf = []
for f_pagina in glob.glob("../data/presidente/mananera/*"):
    urls_conf += [path for _, _, path in re.findall(pattern, open(f_pagina, "r", encoding = "utf-8").read()) if "estenografica-de-la" in path]

In [15]:
urls_conf_set = list(set(urls_conf))
print(len(urls_conf_set))

1018


### Fetch each conference

In [18]:
for path in urls_conf_set:
    download("https://presidente.gob.mx" + path, "../data/presidente/estenograficas/" + path.replace("/", "-"))

### Try to keep text only

In [25]:
for f_pagina in glob.glob("../data/presidente/estenograficas/*"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding = "utf-8").read(), 'html.parser')
    newPath = "../data/presidente/estenograficas_limpias/" + f_pagina.replace("../data/presidente/estenograficas/", "")
    open(newPath, "w", encoding = "utf-8").write(soup.get_text())

### Standardize naming

In [4]:
def normaliza(str_fecha):
    datetimeobject = datetime.strptime(str_fecha[:6] + "20" + str_fecha[6:], '%d.%m.%Y') 
    newformat = datetimeobject.strftime('%Y-%m-%d')
    return newformat

In [5]:
i = 0
fechas_conferencia = []
for f_pagina in glob.glob("../data/presidente/estenograficas_limpias/*"):
    texto = open(f_pagina, 'r', encoding = 'utf-8').read()
    fechas_en_archivo = re.findall('[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]', texto)
    if len(fechas_en_archivo) == 4:
        path = '../data/presidente/estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0])
        if os.path.isfile(path): 
            if not os.path.isfile(path + "_2da"):
                path += "_2da"
                print("Segunda:", f_pagina)
            elif not os.path.isfile(path + "_3ra"):
                path += "_3ra"
                print("Tercera:", f_pagina)
        open(path, 'w', encoding = "utf-8").write(texto)
        fechas_conferencia += [normaliza(fechas_en_archivo[0])]
    else:
        print("Raro:", f_pagina)

Raro: ../data/presidente/estenograficas_limpias/-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-67-
Segunda: ../data/presidente/estenograficas_limpias/-13-05-20-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
Segunda: ../data/presidente/estenograficas_limpias/-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-15-
Segunda: ../data/presidente/estenograficas_limpias/-29-05-19-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-2-
Raro: ../data/presidente/estenograficas_limpias/-29-0920-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
Segunda: ../data/presidente/estenograficas_limpias/-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-21-
Segunda: ../data/presidente/estenografi